# Olist Customer Segmentation Analysis (2017)

## Project Overview
This project presents a comprehensive analysis of Olist's customer base for the year 2017.
The goal is to better understand customer purchasing behavior to inform and improve the company's strategic decisions.

The analysis explores:
- When customers shop (seasonal patterns, weekends)
- What they buy (product categories and quantities)
- Where they are located (geographic distribution)
- How they pay (payment methods and voucher usage)
- Customer satisfaction
- RFM & CLV


## 1. Import Libraries & Load Data

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
customers_df = pd.read_csv('olist_customers_dataset.csv')
geolocation_df = pd.read_csv('olist_geolocation_dataset.csv')
orders_df = pd.read_csv('olist_orders_dataset.csv')
order_items_df = pd.read_csv('olist_order_items_dataset.csv')
order_payments_df = pd.read_csv('olist_order_payments_dataset.csv')
order_reviews_df = pd.read_csv('olist_order_reviews_dataset.csv')
products_df = pd.read_csv('olist_products_dataset.csv')
sellers_df = pd.read_csv('olist_sellers_dataset.csv')
translation_df = pd.read_csv('product_category_name_translation.csv')

## 2. Data Cleaning & Prepration


### 2.1 Checking for Missing Values


In [3]:
dataframes = {
    'customers_df': customers_df,
    'geolocation_df': geolocation_df,
    'orders_df': orders_df,
    'order_items_df': order_items_df,
    'order_payments_df': order_payments_df,
    'order_reviews_df': order_reviews_df,
    'products_df': products_df,
    'sellers_df': sellers_df,
    'translation_df': translation_df
}

for name, df in dataframes.items():
    print(f"--- Missing values in {name} ---")
    missing_values = df.isnull().sum()
    missing_columns = missing_values[missing_values > 0]
    
    if not missing_columns.empty:
        print(missing_columns)
    else:
        print("No missing values found.")
    print("\n")


--- Missing values in customers_df ---
No missing values found.


--- Missing values in geolocation_df ---
No missing values found.


--- Missing values in orders_df ---
order_approved_at                 160
order_delivered_carrier_date     1783
order_delivered_customer_date    2965
dtype: int64


--- Missing values in order_items_df ---
No missing values found.


--- Missing values in order_payments_df ---
No missing values found.


--- Missing values in order_reviews_df ---
review_comment_title      87656
review_comment_message    58247
dtype: int64


--- Missing values in products_df ---
product_category_name         610
product_name_lenght           610
product_description_lenght    610
product_photos_qty            610
product_weight_g                2
product_length_cm               2
product_height_cm               2
product_width_cm                2
dtype: int64


--- Missing values in sellers_df ---
No missing values found.


--- Missing values in translation_df ---
No missing

##### Note: Some columns show "missing values" such as:
- order_approved_at
- order_delivered_carrier_date
- order_delivered_customer_date
These are NOT true missing values.They correspond to orders that were never shipped or not completed ("non-shipped orders").

### 2.2  Handling missing values

In [4]:
products_df['product_category_name'] = products_df['product_category_name'].fillna('not categorized')

In [5]:
not_categorized_count = products_df['product_category_name'].value_counts().get('not categorized', 0)
print("Number of 'not categorized' products:", not_categorized_count)

Number of 'not categorized' products: 610


### 2.3 Merge Products with Category Translations

In [6]:
products = pd.read_csv("olist_products_dataset.csv")
translations = pd.read_csv("product_category_name_translation.csv")
products['product_category_name'] = products['product_category_name'].fillna('not categorized')
products['product_category_name'] = products['product_category_name'].str.strip()
products_merged = products.merge(
    translations,
    on='product_category_name',
    how='left'
)
products_merged['product_category_name_english'] = products_merged['product_category_name_english'].fillna('NN')
print(products_merged.head())

                         product_id  product_category_name  \
0  1e9e8ef04dbcff4541ed26657ea517e5             perfumaria   
1  3aa071139cb16b67ca9e5dea641aaa2f                  artes   
2  96bd76ec8810374ed1b65e291975717f          esporte_lazer   
3  cef67bcfe19066a932b7673e239eb23d                  bebes   
4  9dc1a7de274444849c219cff195d0b71  utilidades_domesticas   

   product_name_lenght  product_description_lenght  product_photos_qty  \
0                 40.0                       287.0                 1.0   
1                 44.0                       276.0                 1.0   
2                 46.0                       250.0                 1.0   
3                 27.0                       261.0                 1.0   
4                 37.0                       402.0                 4.0   

   product_weight_g  product_length_cm  product_height_cm  product_width_cm  \
0             225.0               16.0               10.0              14.0   
1            1000.0     

##### Identify products still without English translation

In [7]:
missing_translations = products_merged[products_merged['product_category_name_english'] == 'NN']

print("Number of products still without translation:", len(missing_translations))
print("Unique original categories still missing translation:")
print(missing_translations['product_category_name'].unique())

Number of products still without translation: 623
Unique original categories still missing translation:
['not categorized' 'pc_gamer'
 'portateis_cozinha_e_preparadores_de_alimentos']


##### Standardize and Manually Correct Translations

In [8]:
# Normalize the column: strip spaces, lowercase
products_merged['product_category_name'] = products_merged['product_category_name'].str.strip().str.lower()


In [9]:
manual_translations = {
    'not categorized': 'unknown',
    'pc_gamer': 'Gaming PC',
    'portateis_cozinha_e_preparadores_de_alimentos': 'Portable Kitchen Appliances & Food Processors'
}


In [10]:
products_merged['product_category_name_english'] = products_merged['product_category_name_english'].replace(manual_translations)


In [11]:
missing_after = products_merged[products_merged['product_category_name_english'] == 'NN']
print("Number of products still without translation:", len(missing_after))


Number of products still without translation: 623


In [12]:
products_merged['product_category_name'] = products_merged['product_category_name'].str.strip().str.lower()

manual_translations = {
    'not categorized': 'unknown',
    'pc_gamer': 'Gaming PC',
    'portateis_cozinha_e_preparadores_de_alimentos': 'Portable Kitchen Appliances & Food Processors'
}

products_merged['product_category_name_english'] = products_merged['product_category_name'].replace(manual_translations)
products_merged['product_category_name_english'] = products_merged['product_category_name_english'].fillna('NN')

missing_after = products_merged[products_merged['product_category_name_english'] == 'NN']
print("Number of products still without translation:", len(missing_after))


Number of products still without translation: 0


In [13]:
products = pd.read_csv("olist_products_dataset.csv")
translations = pd.read_csv("product_category_name_translation.csv")

products['product_category_name'] = products['product_category_name'].fillna('not categorized')
products['product_category_name'] = products['product_category_name'].str.strip().str.lower()

manual_translations = {
    'not categorized': 'unknown',
    'pc_gamer': 'Gaming PC',
    'portateis_cozinha_e_preparadores_de_alimentos': 'Portable Kitchen Appliances & Food Processors'
}

products_merged = products.merge(
    translations,
    on='product_category_name',
    how='left'
)

products_merged['product_category_name_english'] = products_merged['product_category_name'].map(manual_translations)\
    .combine_first(products_merged['product_category_name_english'])\
    .fillna('unknown')

products_merged['product_category_name_english'] = products_merged['product_category_name_english'].str.strip()

category_groups = {
    'computers_accessories': 'Electronics & Technology',
    'computers': 'Electronics & Technology',
    'telephony': 'Electronics & Technology',
    'fixed_telephony': 'Electronics & Technology',
    'consoles_games': 'Electronics & Technology',
    'audio': 'Electronics & Technology',
    'electronics': 'Electronics & Technology',
    'Gaming PC': 'Electronics & Technology',
    'tablets_printing_image': 'Electronics & Technology',
    
    'bed_bath_table': 'Home & Furniture',
    'furniture_decor': 'Home & Furniture',
    'housewares': 'Home & Furniture',
    'kitchen_dining_laundry_garden_furniture': 'Home & Furniture',
    'home_appliances': 'Home & Furniture',
    'home_appliances_2': 'Home & Furniture',
    'home_confort': 'Home & Furniture',
    'home_comfort_2': 'Home & Furniture',
    'small_appliances': 'Home & Furniture',
    'small_appliances_home_oven_and_coffee': 'Home & Furniture',
    'furniture_living_room': 'Home & Furniture',
    'furniture_bedroom': 'Home & Furniture',
    'furniture_mattress_and_upholstery': 'Home & Furniture',
    'air_conditioning': 'Home & Furniture',
    'office_furniture': 'Home & Furniture',
    'Portable Kitchen Appliances & Food Processors': 'Home & Furniture',
    'construction_tools_construction': 'Home & Furniture',
    'costruction_tools_garden': 'Home & Furniture',
    'costruction_tools_tools': 'Home & Furniture',
    'construction_tools_lights': 'Home & Furniture',
    'construction_tools_safety': 'Home & Furniture',
    'home_construction': 'Home & Furniture',
    
    'fashion_bags_accessories': 'Fashion & Accessories',
    'fashion_shoes': 'Fashion & Accessories',
    'fashion_male_clothing': 'Fashion & Accessories',
    'fashio_female_clothing': 'Fashion & Accessories',
    'fashion_underwear_beach': 'Fashion & Accessories',
    'fashion_sport': 'Fashion & Accessories',
    'fashion_childrens_clothes': 'Fashion & Accessories',
    'watches_gifts': 'Fashion & Accessories',
    'luggage_accessories': 'Fashion & Accessories',
    'christmas_supplies': 'Fashion & Accessories',
    
    'food_drink': 'Food & Drink',
    'food': 'Food & Drink',
    'drinks': 'Food & Drink',
    'la_cuisine': 'Food & Drink',
    
    'health_beauty': 'Health & Beauty',
    'perfumery': 'Health & Beauty',
    'baby': 'Health & Beauty',
    'diapers_and_hygiene': 'Health & Beauty',
    'agro_industry_and_commerce': 'Health & Beauty',
    
    'books_technical': 'Books, Music, Movies & Arts',
    'books_imported': 'Books, Music, Movies & Arts',
    'books_general_interest': 'Books, Music, Movies & Arts',
    'cds_dvds_musicals': 'Books, Music, Movies & Arts',
    'dvds_blu_ray': 'Books, Music, Movies & Arts',
    'music': 'Books, Music, Movies & Arts',
    'cine_photo': 'Books, Music, Movies & Arts',
    'art': 'Books, Music, Movies & Arts',
    'musical_instruments': 'Books, Music, Movies & Arts',
    'arts_and_craftmanship': 'Books, Music, Movies & Arts',
    
    'sports_leisure': 'Sports, Leisure & Outdoors',
    'garden_tools': 'Sports, Leisure & Outdoors',
    'pet_shop': 'Sports, Leisure & Outdoors',
    'flowers': 'Sports, Leisure & Outdoors',
    'signaling_and_security': 'Sports, Leisure & Outdoors',
    'toys': 'Sports, Leisure & Outdoors',
    
    'stationery': 'Stationery & Party Supplies',
    'party_supplies': 'Stationery & Party Supplies',
    'cool_stuff': 'Stationery & Party Supplies',
    
    'auto': 'Automotive & Industrial',
    'industry_commerce_and_business': 'Automotive & Industrial',
    'security_and_services': 'Automotive & Industrial',
    
    'market_place': 'Marketplace & Unknown',
    'unknown': 'Marketplace & Unknown',
    'not categorized': 'Marketplace & Unknown'
}

products_merged['category_group'] = products_merged['product_category_name_english'].map(category_groups)

missing_group = products_merged[products_merged['category_group'].isna()]
print("Number of products without a group:", len(missing_group))
print("Unique categories without a group:", missing_group['product_category_name_english'].unique())


Number of products without a group: 0
Unique categories without a group: []


##### Merge Orders with Customer Data

In [14]:
orders_df = pd.merge(orders_df, customers_df, on='customer_id', how='left')
print(orders_df.head())
print(orders_df.shape)

                           order_id                       customer_id  \
0  e481f51cbdc54678b7cc49136f2d6af7  9ef432eb6251297304e76186b10a928d   
1  53cdb2fc8bc7dce0b6741e2150273451  b0830fb4747a6c6d20dea0b8c802d7ef   
2  47770eb9100c2d0c44946d9cf07ec65d  41ce2a54c0b03bf3443c3d931a367089   
3  949d5b44dbf5de918fe9c16f97b45f8a  f88197465ea7920adcdbec7375364d82   
4  ad21c59c0840e6cb83a9ceb5573f8159  8ab97904e6daea8866dbdbc4fb7aad2c   

  order_status order_purchase_timestamp    order_approved_at  \
0    delivered      2017-10-02 10:56:33  2017-10-02 11:07:15   
1    delivered      2018-07-24 20:41:37  2018-07-26 03:24:27   
2    delivered      2018-08-08 08:38:49  2018-08-08 08:55:23   
3    delivered      2017-11-18 19:28:06  2017-11-18 19:45:59   
4    delivered      2018-02-13 21:18:39  2018-02-13 22:20:29   

  order_delivered_carrier_date order_delivered_customer_date  \
0          2017-10-04 19:55:00           2017-10-10 21:25:13   
1          2018-07-26 14:31:00           2018-08

### 2.4  Convert Order Date Columns to Datetime

In [15]:
date_columns = [
    'order_purchase_timestamp',
    'order_approved_at',
    'order_delivered_carrier_date',
    'order_delivered_customer_date'
]

for col in date_columns:
    orders_df[col] = pd.to_datetime(orders_df[col])


### 2.5 Map Customer States to Regions

In [16]:
state_to_region = {
    'AC':'North','AL':'Northeast','AP':'North','AM':'North','BA':'Northeast',
    'CE':'Northeast','DF':'Central-West','ES':'Southeast','GO':'Central-West',
    'MA':'Northeast','MT':'Central-West','MS':'Central-West','MG':'Southeast',
    'PA':'North','PB':'Northeast','PR':'South','PE':'Northeast','PI':'Northeast',
    'RJ':'Southeast','RN':'Northeast','RS':'South','RO':'North','RR':'North',
    'SC':'South','SP':'Southeast','SE':'Northeast','TO':'North'}
customers_df['region'] = customers_df['customer_state'].map(state_to_region)

### 2.6 Filter Orders for 2017

In [17]:
orders_2017_df = orders_df[orders_df['order_purchase_timestamp'].dt.year == 2017]
print(f"Number of orders in 2017: {orders_2017_df.shape[0]}")
print(f"Earliest order in 2017: {orders_2017_df['order_purchase_timestamp'].min()}")
print(f"Latest order in 2017: {orders_2017_df['order_purchase_timestamp'].max()}")

Number of orders in 2017: 45101
Earliest order in 2017: 2017-01-05 11:56:06
Latest order in 2017: 2017-12-31 23:29:31


## 3. Exploratory Data Analysis 

### 3.1 Customer Growth and Temporal Activity Analysis

####  How many unique customers & orders in 2017? 

In [18]:
num_orders_2017 = orders_2017_df.shape[0]
num_unique_customers_2017 = orders_2017_df['customer_unique_id'].nunique()

print(f"Total number of orders in 2017: {num_orders_2017}")
print(f"Total number of unique customers in 2017: {num_unique_customers_2017}")

Total number of orders in 2017: 45101
Total number of unique customers in 2017: 43713


#### What is the average number of orders per customer?

In [19]:
customer_order_counts = orders_2017_df.groupby('customer_unique_id')['order_id'].count()
average_orders_per_customer = customer_order_counts.mean()

print(f"Average number of orders per customer in 2017: {average_orders_per_customer:.2f}")


Average number of orders per customer in 2017: 1.03


#### How does the customer base & orders grow over the months?

In [20]:
orders_2017_sorted = orders_2017_df.sort_values('order_purchase_timestamp')
seen_customers = set()
cumulative_counts = []
for month, group in orders_2017_sorted.groupby(orders_2017_sorted['order_purchase_timestamp'].dt.to_period('M')):
    seen_customers.update(group['customer_unique_id'].unique())
    cumulative_counts.append((month, len(seen_customers)))
cumulative_unique_customers = pd.DataFrame(cumulative_counts, columns=['order_month', 'cumulative_unique_customers'])
print(cumulative_unique_customers)

   order_month  cumulative_unique_customers
0      2017-01                          765
1      2017-02                         2517
2      2017-03                         5153
3      2017-04                         7506
4      2017-05                        11102
5      2017-06                        14241
6      2017-07                        18136
7      2017-08                        22320
8      2017-09                        26451
9      2017-10                        30921
10     2017-11                        38226
11     2017-12                        43713


In [21]:
orders_2017_df_copy = orders_2017_df.copy()
orders_2017_sorted = orders_2017_df_copy.sort_values('order_purchase_timestamp')
monthly_counts = []
for month, group in orders_2017_sorted.groupby(orders_2017_sorted['order_purchase_timestamp'].dt.to_period('M')):
    monthly_counts.append((month, group.shape[0]))
monthly_orders = pd.DataFrame(monthly_counts, columns=['order_month', 'monthly_order_count'])
print("Monthly orders in 2017:")
print(monthly_orders)
total_orders_2017 = orders_2017_df_copy.shape[0]
print(f"\nTotal number of orders in 2017: {total_orders_2017}")

Monthly orders in 2017:
   order_month  monthly_order_count
0      2017-01                  800
1      2017-02                 1780
2      2017-03                 2682
3      2017-04                 2404
4      2017-05                 3700
5      2017-06                 3245
6      2017-07                 4026
7      2017-08                 4331
8      2017-09                 4285
9      2017-10                 4631
10     2017-11                 7544
11     2017-12                 5673

Total number of orders in 2017: 45101


 #### What is the percentage of customers made more than one purchase in 2017?

In [22]:
customer_order_counts = orders_2017_df.groupby('customer_unique_id')['order_id'].count()
repeated_customers = customer_order_counts[customer_order_counts > 1]
num_repeated_customers = repeated_customers.shape[0]
total_unique_customers = orders_2017_df['customer_unique_id'].nunique()
percentage_repeated = (num_repeated_customers / total_unique_customers) * 100

print(f"Number of repeated customers in 2017: {num_repeated_customers}")
print(f"Percentage of repeated customers: {percentage_repeated:.2f}%")

Number of repeated customers in 2017: 1256
Percentage of repeated customers: 2.87%


#### Which product groups in 2017 had customers who placed more than one order within the same Group?"

In [128]:
orders_products_2017 = orders_2017.merge(
    df_order_items.merge(products_merged[['product_id', 'category_group']], on='product_id', how='left'),
    on='order_id',
    how='left'
)

customer_category_orders = orders_products_2017.groupby(
    ['customer_unique_id', 'category_group']
)['order_id'].nunique().reset_index(name='order_count')

repeated_category_customers = customer_category_orders[customer_category_orders['order_count'] > 1]

category_summary = repeated_category_customers.groupby('category_group').agg(
    repeated_customers=('customer_unique_id', 'nunique'),
    total_orders=('order_count', 'sum')
).reset_index()

print(category_summary.sort_values('repeated_customers', ascending=False))


                category_group  repeated_customers  total_orders
6             Home & Furniture                 380           795
8   Sports, Leisure & Outdoors                 153           317
5              Health & Beauty                 118           240
2     Electronics & Technology                  75           153
3        Fashion & Accessories                  71           150
0      Automotive & Industrial                  18            37
9  Stationery & Party Supplies                  11            22
4                 Food & Drink                   5            11
1  Books, Music, Movies & Arts                   4             8
7        Marketplace & Unknown                   3             6


#### How many customers in 2017 purchased from more than one product group, and what percentage do they represent of the total customer base?

In [127]:
orders_2017_with_cust = orders_2017_df.merge(
    df_customers[['customer_id', 'customer_unique_id']],
    on='customer_id',
    how='left'
)


orders_products_2017 = orders_2017_with_cust.merge(
    df_order_items.merge(products_merged[['product_id', 'category_group']], on='product_id', how='left'),
    on='order_id',
    how='left'
)

customer_groups_count = orders_products_2017.groupby('customer_unique_id')['category_group'].nunique()

multi_group_customers = customer_groups_count[customer_groups_count > 1]

num_multi_group_customers = multi_group_customers.shape[0]
total_unique_customers = orders_2017_with_cust['customer_unique_id'].nunique()
percentage_multi_group = (num_multi_group_customers / total_unique_customers) * 100

print(f"Number of customers who purchased from more than one group in 2017: {num_multi_group_customers}")
print(f"Percentage of total customers: {percentage_multi_group:.2f}%")


Number of customers who purchased from more than one group in 2017: 649
Percentage of total customers: 1.48%


#### What is the percentage of seasonal versus regular customers in 2017, where seasonal customers only purchased during seasonal periods and did not place any orders outside these periods?"

In [23]:
df_merged = pd.merge(
    customers_df[['customer_unique_id', 'customer_id']],
    orders_2017_df[['customer_id', 'order_purchase_timestamp']],
    on='customer_id',
    how='inner'
)

seasonal_periods = [
    ('2017-05-07', '2017-05-14', "Mother's Day week"),
    ('2017-06-05', '2017-06-12', "Valentine's week"),
    ('2017-08-06', '2017-08-13', "Father's week"),
    ('2017-10-05', '2017-10-12', "Child's Day week"),
    ('2017-11-20', '2017-11-27', "Black Friday week")
]

seasonal_periods = [(pd.to_datetime(start), pd.to_datetime(end), name) for start, end, name in seasonal_periods]

seasonal_customers = set()
for start_date, end_date, name in seasonal_periods:
    mask = (df_merged['order_purchase_timestamp'] >= start_date) & (df_merged['order_purchase_timestamp'] <= end_date)
    seasonal_customers.update(df_merged.loc[mask, 'customer_unique_id'].unique())

all_customers_2017 = set(df_merged['customer_unique_id'].unique())
regular_customers = all_customers_2017 - seasonal_customers

seasonal_percentage = (len(seasonal_customers) / len(all_customers_2017)) * 100
regular_percentage = (len(regular_customers) / len(all_customers_2017)) * 100

print(f"Total unique seasonal customers in 2017: {len(seasonal_customers)}")
print(f"Total unique regular customers in 2017: {len(regular_customers)}")
print(f"\nPercentage breakdown:")
print(f"Seasonal customers: {seasonal_percentage:.2f}%")
print(f"Regular customers: {regular_percentage:.2f}%")


Total unique seasonal customers in 2017: 6527
Total unique regular customers in 2017: 37186

Percentage breakdown:
Seasonal customers: 14.93%
Regular customers: 85.07%


#### What is the percentage of Weekened versus regular customers in 2017, where Weekend customers only purchased during Weekend period and did not place any orders outside weekends?"

In [24]:
df_merged = orders_2017_df.copy()

df_merged['is_weekend'] = df_merged['order_purchase_timestamp'].dt.weekday >= 5

customer_weekend_behavior = df_merged.groupby('customer_unique_id', as_index=False)['is_weekend'].min()

customer_weekend_behavior['Buyer_Type'] = customer_weekend_behavior['is_weekend'].map({True: 'Weekend Buyer', False: 'Regular Buyer'})

buyer_counts = customer_weekend_behavior['Buyer_Type'].value_counts()
buyer_percentage = customer_weekend_behavior['Buyer_Type'].value_counts(normalize=True) * 100

print("Customer counts by buyer type:")
print(buyer_counts)
print("\nPercentage of customers by buyer type:")
print(buyer_percentage)


Customer counts by buyer type:
Buyer_Type
Regular Buyer    33625
Weekend Buyer    10088
Name: count, dtype: int64

Percentage of customers by buyer type:
Buyer_Type
Regular Buyer    76.922197
Weekend Buyer    23.077803
Name: proportion, dtype: float64


#### How many orders and customers used vouchers in 2017, what proportion of customers used them at least once, who used them exclusively, and who never used them?”

In [25]:
orders_payments_2017 = orders_2017_df.merge(
    order_payments_df[['order_id', 'payment_type']],
    on='order_id',
    how='left'
)

orders_voucher_flags = orders_payments_2017.groupby('order_id')['payment_type'].apply(lambda x: 'voucher' in x.values)
num_orders_with_voucher = orders_voucher_flags.sum()
num_orders_without_voucher = (~orders_voucher_flags).sum()

customer_payment_types = orders_payments_2017.groupby('customer_unique_id')['payment_type'].unique()
customers_only_voucher = customer_payment_types[customer_payment_types.apply(lambda x: set(x) == {'voucher'})]
num_customers_only_voucher = customers_only_voucher.shape[0]

customers_with_voucher = customer_payment_types[customer_payment_types.apply(lambda x: 'voucher' in x)]
num_customers_with_voucher = customers_with_voucher.shape[0]

customers_without_voucher = customer_payment_types[customer_payment_types.apply(lambda x: 'voucher' not in x)]
num_customers_without_voucher = customers_without_voucher.shape[0]

total_customers_2017 = orders_2017_df['customer_unique_id'].nunique()
percentage_only_voucher = (num_customers_only_voucher / total_customers_2017) * 100
percentage_customers_with_voucher = (num_customers_with_voucher / total_customers_2017) * 100
percentage_customers_without_voucher = (num_customers_without_voucher / total_customers_2017) * 100

print(f"Number of orders with vouchers: {num_orders_with_voucher}")
print(f"Number of orders without vouchers: {num_orders_without_voucher}")
print(f"Number of customers who purchased at least once with vouchers: {num_customers_with_voucher}")
print(f"Percentage of customers who purchased at least once with vouchers: {percentage_customers_with_voucher:.2f}%")
print(f"Number of customers who purchased only with vouchers: {num_customers_only_voucher}")
print(f"Percentage of customers who purchased only with vouchers: {percentage_only_voucher:.2f}%")
print(f"Number of customers who never used vouchers: {num_customers_without_voucher}")
print(f"Percentage of customers who never used vouchers: {percentage_customers_without_voucher:.2f}%")


Number of orders with vouchers: 1925
Number of orders without vouchers: 43176
Number of customers who purchased at least once with vouchers: 1882
Percentage of customers who purchased at least once with vouchers: 4.31%
Number of customers who purchased only with vouchers: 713
Percentage of customers who purchased only with vouchers: 1.63%
Number of customers who never used vouchers: 41831
Percentage of customers who never used vouchers: 95.69%


In [26]:
orders_payments_2017 = order_payments_df[order_payments_df['order_id'].isin(orders_2017_df['order_id'])]
payment_counts_per_order = orders_payments_2017.groupby('order_id')['payment_type'].nunique()
orders_multiple_payment_methods = payment_counts_per_order[payment_counts_per_order > 1].count()
print(f"Number of orders with more than one payment method: {orders_multiple_payment_methods}")


Number of orders with more than one payment method: 1169


In [88]:
orders_payments_2017 = order_payments_df[order_payments_df['order_id'].isin(orders_2017_df['order_id'])]
voucher_only_orders = orders_payments_2017.groupby('order_id')['payment_type'].apply(lambda x: set(x) == {'voucher'})
num_voucher_only_orders = voucher_only_orders.sum()
print(f"Number of orders paid only with vouchers: {num_voucher_only_orders}")

Number of orders paid only with vouchers: 756


In [89]:
total_transactions_with_voucher = orders_voucher_flags.sum()
total_transactions_without_voucher = (~orders_voucher_flags).sum()

print(f"Total transactions with voucher: {total_transactions_with_voucher}")
print(f"Total transactions without voucher: {total_transactions_without_voucher}")


Total transactions with voucher: 1925
Total transactions without voucher: 43176


In [90]:
total_voucher_transactions = orders_payments_2017[orders_payments_2017['payment_type'] == 'voucher'].shape[0]

print(f"Total transactions involving vouchers: {total_voucher_transactions}")


Total transactions involving vouchers: 3027


### How many orders in 2017 were purchased at a discount, and what is the distribution of customers who bought discounted products versus those who never bought discounted products?

In [28]:
orders_items_df = pd.read_csv('olist_order_items_dataset.csv')
order_payments_df = pd.read_csv('olist_order_payments_dataset.csv')

orders_items_payments_2017 = orders_2017_df.merge(
    orders_items_df[['order_id', 'price', 'freight_value']],
    on='order_id',
    how='left'
).merge(
    order_payments_df[['order_id', 'payment_value']],
    on='order_id',
    how='left'
)

orders_total_2017 = orders_items_payments_2017.groupby('order_id').agg(
    total_price=('price', 'sum'),
    total_freight=('freight_value', 'sum'),
    total_payment=('payment_value', 'sum'),
    customer_unique_id=('customer_unique_id', 'first')
).reset_index()

orders_total_2017['is_discounted'] = orders_total_2017['total_payment'] < (orders_total_2017['total_price'] + orders_total_2017['total_freight'])

num_discounted_orders = orders_total_2017['is_discounted'].sum()
num_non_discounted_orders = (~orders_total_2017['is_discounted']).sum()

customer_discount_info = orders_total_2017.groupby('customer_unique_id')['is_discounted'].agg(list).reset_index()
customers_discounted_once = customer_discount_info[customer_discount_info['is_discounted'].apply(lambda x: any(x))].shape[0]
customers_only_discounted = customer_discount_info[customer_discount_info['is_discounted'].apply(lambda x: all(x))].shape[0]
customers_never_discounted = customer_discount_info[customer_discount_info['is_discounted'].apply(lambda x: not any(x))].shape[0]

total_customers = orders_total_2017['customer_unique_id'].nunique()

print(f"Number of discounted orders: {num_discounted_orders}")
print(f"Number of non-discounted orders: {num_non_discounted_orders}")
print(f"Total customers who purchased at least once discounted: {customers_discounted_once} ({customers_discounted_once/total_customers*100:.2f}%)")
print(f"Total customers who purchased only discounted: {customers_only_discounted} ({customers_only_discounted/total_customers*100:.2f}%)")
print(f"Total customers who never purchased discounted: {customers_never_discounted} ({customers_never_discounted/total_customers*100:.2f}%)")


Number of discounted orders: 5106
Number of non-discounted orders: 39995
Total customers who purchased at least once discounted: 5063 (11.58%)
Total customers who purchased only discounted: 4882 (11.17%)
Total customers who never purchased discounted: 38650 (88.42%)


### Monthly Distribution of Unique Customers Purchasing Discounted Products in 2017

In [130]:
orders_df = pd.read_csv('olist_orders_dataset.csv')
order_items_df = pd.read_csv('olist_order_items_dataset.csv')
order_payments_df = pd.read_csv('olist_order_payments_dataset.csv')
customers_df = pd.read_csv('olist_customers_dataset.csv')


orders_df['order_purchase_timestamp'] = pd.to_datetime(orders_df['order_purchase_timestamp'])


orders_2017_df = orders_df[orders_df['order_purchase_timestamp'].dt.year == 2017].copy()


merged_df = pd.merge(orders_2017_df, order_items_df, on='order_id', how='inner')


final_merged_df = pd.merge(merged_df, order_payments_df, on='order_id', how='inner')


agg_df = final_merged_df.groupby('order_id').agg(
    total_price=('price', 'sum'),
    total_freight=('freight_value', 'sum'),
    total_payment=('payment_value', 'sum')
).reset_index()


agg_df['is_discounted'] = (agg_df['total_price'] + agg_df['total_freight']) > agg_df['total_payment']


discounted_orders_customers_df = pd.merge(
    agg_df, orders_2017_df, on='order_id', how='inner'
)


discounted_orders_customers_df = pd.merge(
    discounted_orders_customers_df,
    customers_df[['customer_id', 'customer_unique_id']],
    on='customer_id', how='inner'
)


discounted_orders_customers_df = discounted_orders_customers_df[
    discounted_orders_customers_df['is_discounted'] == True
].copy()


discounted_orders_customers_df['month'] = discounted_orders_customers_df['order_purchase_timestamp'].dt.strftime('%b')


monthly_customers = discounted_orders_customers_df.groupby('month')['customer_unique_id'].nunique()


months_order = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
monthly_customers = monthly_customers.reindex(months_order, fill_value=0)


monthly_customers['Total'] = discounted_orders_customers_df['customer_unique_id'].nunique()

print("Number of unique customers who bought discounted products per month in 2017:")
print(monthly_customers)


Number of unique customers who bought discounted products per month in 2017:
month
Jan        93
Feb       243
Mar       336
Apr       326
May       408
Jun       359
Jul       467
Aug       457
Sep       471
Oct       507
Nov       823
Dec       586
Total    5063
Name: customer_unique_id, dtype: int64


### Monthly Distribution of Unique Orders for Discounted Products in 2017

In [133]:

orders_df = pd.read_csv('olist_orders_dataset.csv')
order_items_df = pd.read_csv('olist_order_items_dataset.csv')
order_payments_df = pd.read_csv('olist_order_payments_dataset.csv')
customers_df = pd.read_csv('olist_customers_dataset.csv')

orders_df['order_purchase_timestamp'] = pd.to_datetime(orders_df['order_purchase_timestamp'])

orders_2017_df = orders_df[orders_df['order_purchase_timestamp'].dt.year == 2017].copy()

merged_df = pd.merge(orders_2017_df, order_items_df, on='order_id', how='inner')
final_merged_df = pd.merge(merged_df, order_payments_df, on='order_id', how='inner')

agg_df = final_merged_df.groupby('order_id').agg(
    total_price=('price', 'sum'),
    total_freight=('freight_value', 'sum'),
    total_payment=('payment_value', 'sum')
).reset_index()

agg_df['is_discounted'] = (agg_df['total_price'] + agg_df['total_freight']) > agg_df['total_payment']

discounted_orders_df = pd.merge(
    agg_df, orders_2017_df, on='order_id', how='inner'
)

discounted_orders_df = discounted_orders_df[discounted_orders_df['is_discounted'] == True].copy()

discounted_orders_df['month'] = discounted_orders_df['order_purchase_timestamp'].dt.strftime('%b')

monthly_orders = discounted_orders_df.groupby('month')['order_id'].count()

months_order = ['Jan','Feb','Mar','Apr','May','Jun','Jul','Aug','Sep','Oct','Nov','Dec']
monthly_orders = monthly_orders.reindex(months_order, fill_value=0)

monthly_orders['Total'] = discounted_orders_df['order_id'].count()

print("Number of discounted orders per month in 2017:")
print(monthly_orders)

Number of discounted orders per month in 2017:
month
Jan        97
Feb       243
Mar       341
Apr       327
May       411
Jun       363
Jul       473
Aug       459
Sep       471
Oct       509
Nov       824
Dec       588
Total    5106
Name: order_id, dtype: int64


## 3.2 Geographic Distribution

In [29]:

unique_customers = customers_df.drop_duplicates(subset='customer_unique_id')[['customer_unique_id', 'region']]

unique_customers_2017 = unique_customers.merge(
    orders_2017_df[['customer_id', 'customer_unique_id']].drop_duplicates(),
    on='customer_unique_id',
    how='inner'
)


unique_customers_2017_per_region = unique_customers_2017.groupby('region')['customer_unique_id'].nunique()

print(unique_customers_2017_per_region)
print("Total unique customers in 2017:", unique_customers_2017_per_region.sum())


region
Central-West     2513
North             926
Northeast        4408
South            6452
Southeast       29414
Name: customer_unique_id, dtype: int64
Total unique customers in 2017: 43713


In [39]:

state_to_region = {
    'AC':'North','AL':'Northeast','AP':'North','AM':'North','BA':'Northeast',
    'CE':'Northeast','DF':'Central-West','ES':'Southeast','GO':'Central-West',
    'MA':'Northeast','MT':'Central-West','MS':'Central-West','MG':'Southeast',
    'PA':'North','PB':'Northeast','PR':'South','PE':'Northeast','PI':'Northeast',
    'RJ':'Southeast','RN':'Northeast','RS':'South','RO':'North','RR':'North',
    'SC':'South','SP':'Southeast','SE':'Northeast','TO':'North'
}

customers_df['region'] = customers_df['customer_state'].map(state_to_region)

orders_df['order_purchase_timestamp'] = pd.to_datetime(orders_df['order_purchase_timestamp'])
orders_2017_df = orders_df[orders_df['order_purchase_timestamp'].dt.year == 2017]

orders_with_region = orders_2017_df.merge(
    customers_df[['customer_id', 'region']],
    on='customer_id',
    how='left'
)


orders_count_by_region = orders_with_region.groupby('region')['order_id'].count().reset_index()
orders_count_by_region = orders_count_by_region.sort_values(by='order_id', ascending=False)


total_orders = orders_2017_df['order_id'].nunique()
orders_count_by_region = pd.concat([
    orders_count_by_region,
    pd.DataFrame([{'region': 'Total', 'order_id': total_orders}])
], ignore_index=True)

orders_count_by_region

,region,order_id
0,Southeast,30367
1,South,6651
2,Northeast,4531
3,Central-West,2597
4,North,955
5,Total,45101


## 3.3 Buyer and Product Preferences

### Which product category groups had the highest number of items sold in 2017?

In [40]:
orders_items_2017 = orders_2017_df.merge(
    order_items_df, 
    on='order_id', 
    how='inner'
)

orders_items_with_categories = orders_items_2017.merge(
    products_merged[['product_id', 'category_group']], 
    on='product_id', 
    how='left'
)

orders_items_with_categories['category_group'] = orders_items_with_categories['category_group'].fillna('Marketplace & Unknown')

items_per_category = orders_items_with_categories.groupby('category_group')['order_item_id'].count().reset_index()
items_per_category.columns = ['category_group', 'num_items_sold']
items_per_category = items_per_category.sort_values('num_items_sold', ascending=False)

total_items_2017 = items_per_category['num_items_sold'].sum()
items_per_category['percentage'] = (items_per_category['num_items_sold'] / total_items_2017 * 100).round(2)

items_per_category = pd.concat([
    items_per_category,
    pd.DataFrame([{
        'category_group': 'Total',
        'num_items_sold': total_items_2017,
        'percentage': items_per_category['percentage'].sum()
    }])
], ignore_index=True)

items_per_category


,category_group,num_items_sold,percentage
0,Home & Furniture,15047,29.58
1,"Sports, Leisure & Outdoors",9998,19.66
2,Electronics & Technology,7423,14.59
3,Health & Beauty,6756,13.28
4,Fashion & Accessories,4529,8.90
5,Stationery & Party Supplies,3310,6.51
6,Automotive & Industrial,1641,3.23
7,Marketplace & Unknown,1108,2.18
8,"Books, Music, Movies & Arts",692,1.36
9,Food & Drink,360,0.71


#### How many unique customers purchased single vs. multiple product types in 2017, and what are their proportions?

In [66]:

df_customers = pd.read_csv('olist_customers_dataset.csv')
df_orders = pd.read_csv('olist_orders_dataset.csv')
df_order_items = pd.read_csv('olist_order_items_dataset.csv')


df_orders['order_purchase_timestamp'] = pd.to_datetime(df_orders['order_purchase_timestamp'])


orders_2017 = df_orders[df_orders['order_purchase_timestamp'].dt.year == 2017].copy()


orders_2017 = pd.merge(
    orders_2017,
    df_customers[['customer_id', 'customer_unique_id']],
    on='customer_id',
    how='left'
)


total_customers_2017 = orders_2017['customer_unique_id'].nunique()


orders_items_2017 = pd.merge(
    orders_2017[['order_id', 'customer_unique_id']],
    df_order_items[['order_id', 'product_id']],
    on='order_id',
    how='inner'
)


customer_unique_products = orders_items_2017.groupby('customer_unique_id')['product_id'].nunique()


multi_product_customers = customer_unique_products[customer_unique_products > 1]
single_product_customers = customer_unique_products[customer_unique_products == 1]


other_customers_count = total_customers_2017 - (len(multi_product_customers) + len(single_product_customers))
single_product_total = len(single_product_customers) + other_customers_count

print(f"Total number of unique customers in 2017: {total_customers_2017}")
print(f"Customers who bought multiple product types: {len(multi_product_customers)}")
print(f"Customers who bought a single product type: {single_product_total}")
print(f"Percentage of multi-product customers: {len(multi_product_customers)/total_customers_2017*100:.2f}%")
print(f"Percentage of single-product customers: {single_product_total/total_customers_2017*100:.2f}%")


Total number of unique customers in 2017: 43713
Customers who bought multiple product types: 2409
Customers who bought a single product type: 41304
Percentage of multi-product customers: 5.51%
Percentage of single-product customers: 94.49%


#### How many customers in 2017 placed single-item orders versus multi-item orders, and what are their proportions?

In [70]:

df_customers = pd.read_csv('olist_customers_dataset.csv')
df_orders = pd.read_csv('olist_orders_dataset.csv')
df_order_items = pd.read_csv('olist_order_items_dataset.csv')


df_orders['order_purchase_timestamp'] = pd.to_datetime(df_orders['order_purchase_timestamp'])


orders_2017 = df_orders[df_orders['order_purchase_timestamp'].dt.year == 2017].copy()


orders_2017 = pd.merge(
    orders_2017,
    df_customers[['customer_id', 'customer_unique_id']],
    on='customer_id',
    how='inner'  
)

order_item_counts = df_order_items.groupby('order_id')['order_item_id'].count().reset_index()
order_item_counts.rename(columns={'order_item_id': 'num_items'}, inplace=True)


orders_with_item_count = pd.merge(
    orders_2017[['order_id', 'customer_unique_id']],
    order_item_counts,
    on='order_id',
    how='left'
)


orders_with_item_count['num_items'] = orders_with_item_count['num_items'].fillna(1)


orders_with_item_count['Order_Type'] = orders_with_item_count['num_items'].apply(
    lambda x: 'Single-Item Order' if x == 1 else 'Multi-Item Order'
)


customer_classification = orders_with_item_count.groupby('customer_unique_id')['Order_Type'].apply(
    lambda x: 'Multi-Item Customer' if 'Multi-Item Order' in x.values else 'Single-Item Customer'
).reset_index()

customer_classification.columns = ['customer_unique_id', 'Customer_Type']


customer_counts = customer_classification['Customer_Type'].value_counts()
customer_percentage = customer_classification['Customer_Type'].value_counts(normalize=True) * 100


result_df = pd.DataFrame({
    'Count': customer_counts,
    'Percentage': customer_percentage
})
print(result_df)


print(f"\nSum of customer types: {result_df['Count'].sum()}")

                      Count  Percentage
Customer_Type                          
Single-Item Customer  39330   89.973235
Multi-Item Customer    4383   10.026765

Sum of customer types: 43713


### 3.4 Payment preferences

#### What is the most used payment method by cutomers? (by number of transactions)

In [41]:

orders_payments_2017 = orders_2017_df.merge(
    order_payments_df[['order_id', 'payment_type']],
    on='order_id',
    how='left'
)


transactions_count_2017 = orders_payments_2017['payment_type'].value_counts().reset_index()
transactions_count_2017.columns = ['payment_type', 'num_transactions']


total_transactions = transactions_count_2017['num_transactions'].sum()
transactions_count_2017['percentage'] = (
    transactions_count_2017['num_transactions'] / total_transactions * 100
).round(2)

transactions_count_2017


,payment_type,num_transactions,percentage
0,credit_card,34568,72.74
1,boleto,9508,20.01
2,voucher,3027,6.37
3,debit_card,422,0.89


#### Excluding Voucher as it is a Gift not payment method.

In [71]:

orders_payments_2017 = orders_2017_df.merge(
    order_payments_df[['order_id', 'payment_type']],
    on='order_id',
    how='left'
)


orders_payments_2017 = orders_payments_2017[orders_payments_2017['payment_type'] != 'voucher']


transactions_count_2017 = orders_payments_2017['payment_type'].value_counts().reset_index()
transactions_count_2017.columns = ['payment_type', 'num_transactions']


total_transactions = transactions_count_2017['num_transactions'].sum()
transactions_count_2017['percentage'] = (
    transactions_count_2017['num_transactions'] / total_transactions * 100
).round(2)

transactions_count_2017


,payment_type,num_transactions,percentage
0,credit_card,34568,77.68
1,boleto,9508,21.37
2,debit_card,422,0.95


## 3.5 Satisfaction-based Segmentation

In [43]:
orders_2017_df = orders_df[orders_df['order_purchase_timestamp'].dt.year == 2017].copy()

df_merged = pd.merge(
    customers_df[['customer_unique_id', 'customer_id']],
    orders_2017_df[['order_id', 'customer_id']],
    on='customer_id',
    how='inner'
)

df_merged = pd.merge(
    df_merged,
    order_reviews_df[['order_id', 'review_score']],
    on='order_id',
    how='left'
)

customer_review_summary = df_merged.groupby('customer_unique_id')['review_score'].mean().reset_index()
customer_review_summary.rename(columns={'review_score': 'average_review_score'}, inplace=True)

def classify_review_segment(average_score):
    if average_score == 5.0:
        return 'Happy Customer'
    elif average_score >= 3.0:
        return 'Neutral Customer'
    else:
        return 'Unhappy Customer'

customer_review_summary['Review_Segment'] = customer_review_summary['average_review_score'].apply(classify_review_segment)

review_segment_counts = customer_review_summary['Review_Segment'].value_counts()
review_segment_percentage = customer_review_summary['Review_Segment'].value_counts(normalize=True) * 100

review_segment_summary = pd.DataFrame({
    'Segment': review_segment_counts.index,
    'Num_Customers': review_segment_counts.values,
    'Percentage': review_segment_percentage.values.round(2)
})

review_segment_summary = review_segment_summary.sort_values(by='Num_Customers', ascending=False).reset_index(drop=True)

review_segment_summary

,Segment,Num_Customers,Percentage
0,Happy Customer,24643,56.37
1,Neutral Customer,12435,28.45
2,Unhappy Customer,6635,15.18


### What is the "delay in delivery" percentage among satisfaction segments?

In [45]:
orders_df['order_purchase_timestamp'] = pd.to_datetime(orders_df['order_purchase_timestamp'])

orders_2017_df = orders_df[orders_df['order_purchase_timestamp'].dt.year == 2017].copy()

df_merged = pd.merge(
    customers_df[['customer_unique_id', 'customer_id']],
    orders_2017_df[['order_id', 'customer_id', 'order_delivered_customer_date', 'order_estimated_delivery_date']],
    on='customer_id',
    how='inner'
)

df_merged = pd.merge(
    df_merged,
    order_reviews_df[['order_id', 'review_score']],
    on='order_id',
    how='left'
)

customer_review_summary = df_merged.groupby('customer_unique_id')['review_score'].mean().reset_index()
customer_review_summary.rename(columns={'review_score': 'average_review_score'}, inplace=True)

def classify_review_segment(average_score):
    if average_score == 5.0:
        return 'Happy Customer'
    elif average_score >= 3.0:
        return 'Neutral Customer'
    else:
        return 'Unhappy Customer'

customer_review_summary['Review_Segment'] = customer_review_summary['average_review_score'].apply(classify_review_segment)

df_analysis_2017 = pd.merge(
    orders_2017_df[['order_id', 'customer_id', 'order_delivered_customer_date', 'order_estimated_delivery_date']],
    order_reviews_df[['order_id', 'review_score']],
    on='order_id',
    how='left'
)

df_analysis_2017 = pd.merge(
    df_analysis_2017,
    customers_df[['customer_unique_id', 'customer_id']],
    on='customer_id',
    how='left'
)

df_analysis_2017 = pd.merge(
    df_analysis_2017,
    customer_review_summary[['customer_unique_id', 'Review_Segment']],
    on='customer_unique_id',
    how='left'
)

df_analysis_2017['order_delivered_customer_date'] = pd.to_datetime(df_analysis_2017['order_delivered_customer_date'])
df_analysis_2017['order_estimated_delivery_date'] = pd.to_datetime(df_analysis_2017['order_estimated_delivery_date'])

df_analysis_2017['is_late'] = df_analysis_2017['order_delivered_customer_date'] > df_analysis_2017['order_estimated_delivery_date']

late_delivery_percentage_2017 = df_analysis_2017.groupby('Review_Segment')['is_late'].mean() * 100

print("Percentage of late deliveries for each review-based segment (2017):")
print(late_delivery_percentage_2017.round(2))

Percentage of late deliveries for each review-based segment (2017):
Review_Segment
Happy Customer       2.37
Neutral Customer     5.23
Unhappy Customer    23.37
Name: is_late, dtype: float64


### What are the most common reasons written by unhappy customers in the reviews?

In [50]:
from nltk.corpus import stopwords
import re
from collections import Counter


orders_df['order_purchase_timestamp'] = pd.to_datetime(orders_df['order_purchase_timestamp'])
orders_2017 = orders_df[orders_df['order_purchase_timestamp'].dt.year == 2017].copy()


df_merged = pd.merge(customers_df[['customer_unique_id', 'customer_id']],
                     orders_2017[['order_id', 'customer_id']],
                     on='customer_id', how='inner')
df_merged = pd.merge(df_merged,
                     order_reviews_df[['order_id', 'review_score', 'review_comment_message', 'review_comment_title']],
                     on='order_id', how='left')


customer_review_avg = df_merged.groupby('customer_unique_id')['review_score'].mean().reset_index()
unhappy_customers = customer_review_avg[customer_review_avg['review_score'] < 3]['customer_unique_id']


unhappy_reviews = df_merged[df_merged['customer_unique_id'].isin(unhappy_customers)]
unhappy_reviews = unhappy_reviews[unhappy_reviews['review_comment_message'].notna()]

print(f"Number of unhappy customers: {len(unhappy_customers)}")
print(f"Number of reviews with comments from unhappy customers: {len(unhappy_reviews)}")


portuguese_stopwords = set(stopwords.words('portuguese'))


def extract_keywords(text):
    text = str(text).lower()
    text = re.sub(r'[^\w\s]', '', text)  # remove punctuation
    text = re.sub(r'\d+', '', text)      # remove numbers
    words = text.split()
    keywords = [w for w in words if w not in portuguese_stopwords and len(w) > 2]
    return keywords


complaint_patterns = {
    'Delivery Issues': [
        r'atraso', r'demora', r'entrega', r'chegou atrasado', r'demorou na entrega', r'tempo de entrega'
    ],
    'Product Quality': [
        r'quebrado', r'danificado', r'defeito', r'qualidade', r'produto ruim', r'produto defeituoso'
    ],
    'Wrong Item': [
        r'errado', r'diferente', r'cor errada', r'tamanho errado', r'modelo errado'
    ],
    'Missing Parts': [
        r'falta', r'peça', r'acessório', r'incompleto', r'faltando'
    ],
    'Customer Service': [
        r'atendimento', r'suporte', r'resposta lenta', r'não respondeu'
    ],
    'Pricing Issues': [
        r'caro', r'preço', r'custo', r'caríssimo'
    ],
    'Description Mismatch': [
        r'descrição', r'foto', r'imagem', r'informação errada'
    ]
}


def categorize_review(text):
    text = text.lower()
    for category, patterns in complaint_patterns.items():
        for pattern in patterns:
            if re.search(pattern, text):
                return category
    return None  # unmatched reviews will be ignored


category_counter = Counter()
for review in unhappy_reviews['review_comment_message']:
    category = categorize_review(review)
    if category:
        category_counter[category] += 1

# Print results
total_count = sum(category_counter.values())
print("\n" + "="*80)
print("COMPLAINT CATEGORIES ANALYSIS")
print("="*80)
for category, count in category_counter.most_common():
    percentage = (count / total_count) * 100
    print(f"{category}: {count} complaints ({percentage:.1f}%)")

Number of unhappy customers: 6266
Number of reviews with comments from unhappy customers: 4731

COMPLAINT CATEGORIES ANALYSIS
Delivery Issues: 845 complaints (47.2%)
Product Quality: 358 complaints (20.0%)
Missing Parts: 212 complaints (11.8%)
Wrong Item: 205 complaints (11.4%)
Description Mismatch: 80 complaints (4.5%)
Customer Service: 46 complaints (2.6%)
Pricing Issues: 46 complaints (2.6%)


### 3.4 RFM Segmentation

##### In our 2017 customer data analysis, 97% of customers made only one purchase. In such low-frequency environments, monetary value is essentially the same as frequency, adds no additional insight, and may introduce noise and outliers that reduce segmentation accuracy. Therefore, we relied only on Frequency and Recency

##### We will not use (pd.cut) and preferred the conditional (if) method because most customers made only one purchase, so Frequency add no meaningful variation, and the conditional approach provides a clear and interpretable classification based on Recency.

##### Loyal: recent purchase (≤ 90 days) with multiple orders (≥ 2).
##### At Risk: last purchase within half a year (≤ 180 days) but with lower activity.
##### Churned: no purchases for a long time (more than 180 days).

In [51]:
import pandas as pd


orders_df = pd.read_csv('olist_orders_dataset.csv')
customers_df = pd.read_csv('olist_customers_dataset.csv')


orders_df = pd.merge(
    orders_df, 
    customers_df[['customer_id', 'customer_unique_id']], 
    on='customer_id', 
    how='left'
)


orders_df['order_purchase_timestamp'] = pd.to_datetime(orders_df['order_purchase_timestamp'])
orders_2017 = orders_df[orders_df['order_purchase_timestamp'].dt.year == 2017]


rfm = orders_2017.groupby('customer_unique_id').agg({
    'order_purchase_timestamp': lambda x: (pd.Timestamp('2017-12-31') - x.max()).days,
    'order_id': 'count'
}).reset_index()
rfm.columns = ['customer_unique_id', 'Recency', 'Frequency']


def rfm_simple_segment(row):
    if row['Recency'] <= 90 and row['Frequency'] >= 2:  # recent and frequent
        return 'Loyal'
    elif row['Recency'] <= 180 and row['Frequency'] >= 1:  # somewhat recent
        return 'At Risk'
    else:
        return 'Churned'

rfm['Segment'] = rfm.apply(rfm_simple_segment, axis=1)


segment_summary = rfm['Segment'].value_counts().reset_index()
segment_summary.columns = ['Segment','Num_Customers']
segment_summary['Percentage'] = (segment_summary['Num_Customers'] / segment_summary['Num_Customers'].sum() * 100).round(2)

print("Customer Segment Distribution:\n")
print(segment_summary)


Customer Segment Distribution:

   Segment  Num_Customers  Percentage
0  At Risk          28971       66.28
1  Churned          14192       32.47
2    Loyal            550        1.26


### What is the level of satisfaction for each RFM segment?

In [64]:
import pandas as pd


orders_df = pd.read_csv('olist_orders_dataset.csv')
customers_df = pd.read_csv('olist_customers_dataset.csv')
reviews_df = pd.read_csv('olist_order_reviews_dataset.csv')


orders_df = pd.merge(
    orders_df,
    customers_df[['customer_id', 'customer_unique_id']],
    on='customer_id',
    how='left'
)


orders_df['order_purchase_timestamp'] = pd.to_datetime(orders_df['order_purchase_timestamp'])
orders_2017 = orders_df[orders_df['order_purchase_timestamp'].dt.year == 2017]


rfm = orders_2017.groupby('customer_unique_id').agg({
    'order_purchase_timestamp': lambda x: (pd.Timestamp('2017-12-31') - x.max()).days,
    'order_id': 'count'
}).reset_index()
rfm.columns = ['customer_unique_id', 'Recency', 'Frequency']

def rfm_simple_segment(row):
    if row['Recency'] <= 90 and row['Frequency'] >= 2:
        return 'Loyal'
    elif row['Recency'] <= 180 and row['Frequency'] >= 1:
        return 'At Risk'
    else:
        return 'Churned'

rfm['Segment'] = rfm.apply(rfm_simple_segment, axis=1)

segment_reviews = pd.merge(
    rfm[['customer_unique_id', 'Segment']],
    reviews_df[['order_id', 'review_score']],
    left_on='customer_unique_id',
    right_on=orders_2017.set_index('order_id')['customer_unique_id'].reindex(reviews_df['order_id']).values,
    how='left'
)

average_reviews = segment_reviews.groupby('Segment')['review_score'].mean().reset_index()
average_reviews.columns = ['Segment', 'Average_Review']

print(average_reviews)

   Segment  Average_Review
0  At Risk        4.081761
1  Churned        4.092516
2    Loyal        4.155200


### 3.5 CLV (Customer Life Time Value)

In [84]:

orders = orders.merge(customers[['customer_id','customer_unique_id']], on='customer_id')
orders['order_purchase_timestamp'] = pd.to_datetime(orders['order_purchase_timestamp'])
orders_2017 = orders[orders['order_purchase_timestamp'].dt.year == 2017]

order_payments_2017 = payments.merge(
    orders_2017[['order_id','customer_unique_id','order_purchase_timestamp']],
    on='order_id'
)

revenue = order_payments_2017.groupby('customer_unique_id')['payment_value'].sum()
cust_orders = orders_2017.groupby('customer_unique_id').agg(
    total_orders=('order_id','count'),
    first_order=('order_purchase_timestamp','min'),
    last_order=('order_purchase_timestamp','max')
)
cust_orders['lifetime_days'] = (cust_orders['last_order']-cust_orders['first_order']).dt.days+1
cust_orders['avg_order_value'] = revenue / cust_orders['total_orders']
cust_orders['purchase_freq'] = cust_orders['total_orders']/(cust_orders['lifetime_days']/365)
cust_orders['ltv'] = cust_orders['avg_order_value']*cust_orders['purchase_freq']*(cust_orders['lifetime_days']/365)*0.35
cust_orders = cust_orders.replace([np.inf,-np.inf],np.nan).dropna(subset=['ltv'])

print(f"Average LTV: R$ {cust_orders['ltv'].mean():.2f}")
print(f"Median LTV: R$ {cust_orders['ltv'].median():.2f}")
print(f"Total Revenue 2017: R$ {revenue.sum():,.2f}")
print(f"Unique Customers: {len(cust_orders)}")
print(f"Avg Orders/Customer: {cust_orders['total_orders'].mean():.2f}")
print(f"Top 10% LTV: R$ {cust_orders['ltv'].quantile(0.90):.2f}")
print(f"Avg Lifetime (days): {cust_orders['lifetime_days'].mean():.1f}")


Average LTV: R$ 58.05
Median LTV: R$ 37.18
Total Revenue 2017: R$ 7,249,746.73
Unique Customers: 43713
Avg Orders/Customer: 1.03
Top 10% LTV: R$ 111.73
Avg Lifetime (days): 2.3


## Key Insights

#### Customer Base & Purchasing Patterns
- 97% of customers made only **one purchase in 2017** (avg. orders per customer = 1.03).  
- Sales heavily concentrated in the **Southeast region (67.3%)**, especially São Paulo.  
- **Seasonal shopping** (Black Friday, Mother’s Day) accounted for 15% of purchases; 23% occurred exclusively on weekends.

#### Product Preferences
- **Home & Furniture** dominated (30%), followed by **Sports & Leisure** (19.7%) and **Electronics & Technology** (14.6%).  
- 94.5% of customers bought from **only one category**, and 90% placed **single-item orders**.

#### Payment Behavior
- **Credit cards** were used in 77.7% of transactions.  
- **Boleto (bank slips)** accounted for 21.4%.  

#### Customer Satisfaction
- 56% of customers rated their experience as **“Happy” (5 stars)**.  
- Delivery issues caused 47% of complaints, the main driver of dissatisfaction.

#### Customer Health & Retention (RFM Analysis)
- **At Risk**: 66.3%  
- **Churned**: 32.5%  
- **Loyal**: 1.3%  
- Satisfaction scores were similar across segments, suggesting  loyalty is influenced by factors beyond satisfaction .

#### Customer Lifetime Value (CLV)
- Median CLV = R$ 37.18
  
- Top 10% of customers achieved R$ 111.73, highlighting the high value of top segments.


## Recommendations
- **Improve Customer Retention:** Target "At Risk" customers with personalized promotions.  
- **Optimize Delivery:** Focus on reducing delays, as delivery issues are the main cause of dissatisfaction.  
- **Geographic Expansion:** Explore marketing opportunities outside the Southeast region to diversify the customer base.  
- **Encourage Multi-category Purchases:** Incentivize customers to explore more product categories to increase average order value.
---

## Tools & Technologies
- Python
- Tableau
- Excel


## References 

Brazil Help. (n.d.). Brazilian states: Codes and regions. Retrieved September 19, 2025, from https://brazil-help.com/brazilian_states.htm

  
## Contributors
- Mona Mohamed
- Rabab Ali
- Ali Jafar
- Hasan Ali